<a href="https://colab.research.google.com/github/albertpimentel/213-final-proj/blob/main/User_End_Nets213_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries#

In [ ]:
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import re
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Data Preprocessing#

In [ ]:
# drop all rows that were used in mturk
org_df = pd.read_csv('/content/drive/MyDrive/okcupid_profiles.csv') 
mturk_df = pd.read_csv('/content/drive/MyDrive/AGG_Output_Averages.csv') 

In [ ]:
profiles_df = org_df.drop(['essay0', 'essay1','essay2', 'essay3','essay4', 'essay5','essay6', 'essay7','essay8', 'essay9', 'income', 'last_online', 'status', 'ethnicity', 'height', 'location', 'speaks'], axis=1)


In [ ]:
def categorize_bodytype(data_frame):
  for row_index in range(data_frame.shape[0]):
      # Categorize body type
      cell_value = data_frame['body_type'][row_index]
      category = 'average'
      if type(cell_value) != float and 'athletic' in cell_value:
        category = 'fit'
      if type(cell_value) != float and 'skinny' in cell_value:
        category = 'thin'
      if type(cell_value) != float and 'jacked' in cell_value:
        category = 'fit'
      if type(cell_value) != float and 'full figured' in cell_value:
        category = 'curvy'
      if type(cell_value) != float and 'a little extra' in cell_value:
        category = 'curvy'
      data_frame['body_type'][row_index]= category


def categorize_diet(data_frame):
  for row_index in range(data_frame.shape[0]):
      # Categorize diet
      cell_value = data_frame['diet'][row_index]
      category = 'anything'
      if type(cell_value) != float and 'vegetarian' in cell_value:
        category = 'vegetarian'
      if type(cell_value) != float and 'vegan' in cell_value:
        category = 'vegan'
      if type(cell_value) != float and 'halal' in cell_value:
        category = 'halal'
      data_frame['diet'][row_index]= category


def categorize_religion(data_frame):
  for row_index in range(data_frame.shape[0]):
      # Categorize religion
      cell_value = data_frame['religion'][row_index]
      category = 'other'
      if type(cell_value) != float and 'agnostic' in cell_value:
        category = 'agnostic'
      if type(cell_value) != float and 'atheist' in cell_value:
        category = 'atheist'
      if type(cell_value) != float and 'christian' in cell_value:
        category = 'christian'
      if type(cell_value) != float and 'catholicism' in cell_value:
        category = 'catholicism'
      if type(cell_value) != float and 'buddhism' in cell_value:
        category = 'buddhism'
      if type(cell_value) != float and 'hinduism' in cell_value:
        category = 'hinduism'
      if type(cell_value) != float and 'judaism' in cell_value:
        category = 'judaism'
      if type(cell_value) != float and 'islam' in cell_value:
        category = 'islam'
      data_frame['religion'][row_index]= category


def categorize_sign(data_frame):
  for row_index in range(data_frame.shape[0]):
      # Categorize sign
      cell_value = data_frame['sign'][row_index]
      category = 'aquarius'
      if type(cell_value) != float and 'pisces' in cell_value:
        category = 'pisces'
      if type(cell_value) != float and 'aries' in cell_value:
        category = 'aries'
      if type(cell_value) != float and 'taurus' in cell_value:
        category = 'taurus'
      if type(cell_value) != float and 'gemini' in cell_value:
        category = 'gemini'
      if type(cell_value) != float and 'cancer' in cell_value:
        category = 'cancer'
      if type(cell_value) != float and 'leo' in cell_value:
        category = 'leo'
      if type(cell_value) != float and 'virgo' in cell_value:
        category = 'virgo'
      if type(cell_value) != float and 'libra' in cell_value:
        category = 'libra'
      if type(cell_value) != float and 'scorpio' in cell_value:
        category = 'scorpio'
      if type(cell_value) != float and 'sagittarius' in cell_value:
        category = 'sagittarius'
      if type(cell_value) != float and 'capricorn' in cell_value:
        category = 'capricorn'
      data_frame['sign'][row_index]= category  


def categorize_education(data_frame):
  for row_index in range(data_frame.shape[0]):
      # Categorize education
      cell_value = data_frame['education'][row_index]
      category = 'graduated college'
      if type(cell_value) != float and 'dropped out of high school' in cell_value:
        category = 'less than high school'
      if type(cell_value) != float and 'working on high school' in cell_value:
        category = 'less than high school'
      if type(cell_value) != float and 'high school' in cell_value:
        category = 'less than high school'
      if type(cell_value) != float and 'graduated from high school' in cell_value:
        category = 'graduated high school'
      if type(cell_value) != float and 'dropped out of college/university' in cell_value:
        category = 'graduated high school'
      if type(cell_value) != float and 'dropped out of two-year college' in cell_value:
        category = 'graduated high school'
      if type(cell_value) != float and 'working on college/university' in cell_value:
        category = 'in college'
      if type(cell_value) != float and 'working on two-year college' in cell_value:
        category = 'in college'
      if type(cell_value) != float and 'college/university' in cell_value:
        category = 'in college'
      if type(cell_value) != float and 'two-year college' in cell_value:
        category = 'in college'
      data_frame['education'][row_index]= category


# New categorize function
def categorize_new(data_frame):
  categorize_bodytype(data_frame)
  categorize_diet(data_frame)
  categorize_religion(data_frame)
  categorize_sign(data_frame)
  categorize_education(data_frame)


In [ ]:
# Use mturk_df (AGG_Output_Averages.csv) to populate goldstandard_df
goldstandard_data = []
for row_index in range(mturk_df.shape[0]):
  max_average = mturk_df['Max_average'][row_index]
  # Find which rated person has the highest average rating for this profile, arbitarily choosing
  # a profile in case of repeats (since profiles were already randomly selected)
  highest_match_index = -1
  for person_index in range(9):
    column_name = "Answer.comparison" + str(person_index + 1)
    current_rating = mturk_df[column_name][row_index]
    if current_rating == max_average:
      highest_match_index = person_index + 1
      break
  # Add appropriate cells from input profile and corresponding match profile
  # into row of new dataframe
  row_values = []
  attribute_list = ['age', 'sex', 'orientation', 'body_type', 'diet', 'drinks', 'drugs', 'education',
                    'job', 'offspring', 'pets', 'religion', 'sign', 'smokes']
  for attribute in attribute_list:
    row_values.append(mturk_df["Input." + attribute][row_index])
  for attribute in attribute_list:
    row_values.append(mturk_df["Input.person" + str(highest_match_index) + "_" + attribute][row_index])
  goldstandard_data.append(row_values)

goldstandard_df = pd.DataFrame(data = goldstandard_data, columns = [
                    'Input.age', 'Input.sex', 'Input.orientation', 'Input.body_type', 'Input.diet', 'Input.drinks', 'Input.drugs', 'Input.education',
                    'Input.job', 'Input.offspring', 'Input.pets', 'Input.religion', 'Input.sign', 'Input.smokes',
                    'Match.age', 'Match.sex', 'Match.orientation', 'Match.body_type', 'Match.diet', 'Match.drinks', 'Match.drugs', 'Match.education',
                    'Match.job', 'Match.offspring', 'Match.pets', 'Match.religion', 'Match.sign', 'Match.smokes'
                    ])

goldstandard_df.to_csv('/content/drive/MyDrive/goldstandard_data.csv', index=False)

In [ ]:
goldstandard_df.head()
input_df = goldstandard_df[['Input.age', 'Input.sex', 'Input.orientation', 'Input.body_type', 'Input.diet', 'Input.drinks', 'Input.drugs', 'Input.education',
                    'Input.job', 'Input.offspring', 'Input.pets', 'Input.religion', 'Input.sign', 'Input.smokes']]
matches_df = goldstandard_df.drop(['Input.age', 'Input.sex', 'Input.orientation', 'Input.body_type', 'Input.diet', 'Input.drinks', 'Input.drugs', 'Input.education',
                    'Input.job', 'Input.offspring', 'Input.pets', 'Input.religion', 'Input.sign', 'Input.smokes'], axis = 1)                  

In [ ]:
input_df.columns = list(profiles_df.columns)
matches_df.columns = list(profiles_df.columns)

In [ ]:
profiles_df['input_row'] = 0
input_df['input_row'] = 1
matches_df['input_row'] = 2
profiles_df = profiles_df.append(input_df, ignore_index=True)
profiles_df = profiles_df.append(matches_df, ignore_index=True)
categorize_new(profiles_df)





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/

In [ ]:
with_sex_df = profiles_df.copy()

In [ ]:
profiles_df.drop('sex', axis =1, inplace = True)

In [ ]:
data_df = profiles_df.copy()

In [ ]:
# encode trainning and inputs and labels 
from sklearn import preprocessing
cats = list(profiles_df.columns)

for col in cats:
  data_df[col] = data_df[col].astype('category')
  colum = pd.get_dummies(data_df[col], prefix=col)
  data_df = data_df.drop(columns = [col])
  data_df = pd.concat([data_df, colum], axis=1)



In [ ]:
input_encoding = data_df.loc[data_df['input_row_1'] == 1]
labels_encoding = data_df.loc[data_df['input_row_2'] == 1]
train_encoding = data_df.loc[data_df['input_row_0'] == 1]

In [ ]:
input_encoding.drop(['input_row_0','input_row_1','input_row_2'] ,
  axis='columns', inplace=True)
labels_encoding.drop(['input_row_0','input_row_1','input_row_2'] ,
  axis='columns', inplace=True)
train_encoding.drop(['input_row_0','input_row_1','input_row_2'] ,
  axis='columns', inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#Machine Learning Model and Analysis#

In [ ]:
nneigh = NearestNeighbors()
nneigh.fit(train_encoding.to_numpy())


NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
def candidates(df, input):
  df = df.iloc[1: , :]
  if input[2] == 'straight':
    output = df.loc[df['orientation'].isin(['straight', 'bisexual'])]
    output = output.loc[output['sex'] != input[1]]
  elif input[2] == 'gay': 
    output = df.loc[df['orientation'].isin(['bisexual','gay'])]
    output = output.loc[output['sex'] == input[1]]
  else :
    output = df.loc[sex['orientation'].isin(['bisexual','gay','straight'])]
  match = output.head(1)  
  return match    

In [ ]:

def accuracy(input,match):
  i = set(input)
  m = set(match)
  z = len(i.difference(m)) - 2
  if z < 0 :
    z = 0
  return ((len(i) - z)/len(i)) *100


In [ ]:
 def findMatch(in_prf, in_df): 
  profiles_df = org_df.drop(['essay0', 'essay1','essay2', 'essay3','essay4', 'essay5','essay6', 'essay7','essay8', 'essay9', 'income', 'last_online', 'status', 'ethnicity', 'height', 'location', 'speaks'], axis=1)
  x_df = with_sex_df.copy()
  x_df['input_row'] = 0
  matches_df['input_row'] = 0
  x_df = x_df.append(in_df, ignore_index=True)
  x_df = x_df.append(matches_df, ignore_index=True)
 
 
  x_df.drop('sex', axis = 1, inplace = True)
  categorize_new(x_df)


  cats = list(x_df.columns)
  for col in cats:
    x_df[col] = x_df[col].astype('category')
    colum = pd.get_dummies(x_df[col], prefix=col)
    x_df = x_df.drop(columns = [col])
    x_df = pd.concat([x_df, colum], axis=1)
  
  
  input_encoding = x_df.loc[x_df['input_row_1'] == 1]
  input_encoding.drop(['input_row_0','input_row_1'] ,
  axis='columns', inplace=True)
  input_encoding = input_encoding.iloc[: , :-1]

  y_pred = nneigh.kneighbors(input_encoding.to_numpy(), 50, return_distance=False)
  input_profile = in_prf  
  pred_match = profiles_df.iloc[y_pred[0]]
  sex = with_sex_df.iloc[y_pred[0]]    
  m = candidates(sex,input_profile).values.tolist()  
  match = m[0]
  res = round(accuracy(input_profile,match),2)
  return match, res



#Find your Match!#

In [ ]:
input_prf = []

age = input("Your age: ")
input_prf.append(age)
sex = input("Your sex: ")
input_prf.append(sex)
orientation = input("Your orientation: ")
input_prf.append(orientation)
body_type = input("Your body type: ")
input_prf.append(body_type)
diet = input("Your diet: ")
input_prf.append(diet)
drinks = input("Your drinks?: ")
input_prf.append(drinks)
drugs = input("do you do drugs?: ")
input_prf.append(drugs)
education	= input("Whats your education: ") 
input_prf.append(education)
job	 = input("Whats your job: ")
input_prf.append(job)
offspring	= input("Do you want kids: ")
input_prf.append(offspring)
pets	= input("Do you have pets: ")
input_prf.append(pets)
religion	= input("What's your religion: ")
input_prf.append(religion)
sign	= input("Whats'sign : ")
input_prf.append(sign)
smokes	= input("Do you smoke: ")
input_prf.append(smokes)
input_prf.append(9)

# input_prf = ['57', 'm', 'straight', 'fit', 'vegan', 'socially', 'never', 'college/university', 'student', 'wants kids', 'likes cats','agnosticism', 'cancer','no', 9 ]
in_df = pd.DataFrame([input_prf], columns=list(with_sex_df.columns))    
in_df['input_row'] = 1

match, score = findMatch(input_prf, in_df)
match_df = pd.DataFrame([match], columns=list(with_sex_df.columns))
match_df = match_df.iloc[: , :-1]
match_df['score'] = score
match_df.head()


Your age: 22
Your sex: f
Your orientation: straight
Your body type: fit
Your diet: vegan
Your drinks?: socially
do you do drugs?: never
Whats your education: college/university
Whats your job: student
Do you want kids: wants kids
Do you have pets: likes cats
What's your religion: agnosticism
Whats'sign : cancer
Do you smoke: no


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:84: SettingWithCopyWarn

,age,sex,orientation,body_type,diet,drinks,drugs,education,job,offspring,pets,religion,sign,smokes,score
0,22,m,straight,average,vegetarian,socially,often,graduated college,NaN,NaN,likes dogs and likes cats,other,capricorn,sometimes,26.67
